In [1]:
from smolagents import CodeAgent, InferenceClientModel, Tool, tool
from dotenv import load_dotenv
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langfuse import get_client
from huggingface_hub import notebook_login
notebook_login()
load_dotenv()
angfuse_key = os.getenv('ANGFUSE_SECRET_KEY')

In [2]:
class RecipeInfoRetrieverTool(Tool):
    name = "RecipeInfoRetriever"
    description = "Retrieves information about recipes including ingredients, title and instructions based on a natural language query."
    inputs = {
        "query": {
            "type": "string",
            "description": "Natural language query about ingredients or dietary preferences (e.g., 'eggplant and garlic', 'vegan pasta')."
        }
    }
    output_type = "string"
    
    def __init__(self, k: int = 3, **kwargs):
        """Initialize the recipe retrieval tool.
        
        Args:
            k: Number of top recipes to retrieve (default: 3)
        """
        super().__init__(**kwargs)
        self.k = k
        
        print("Loading recipe retrieval system...")
        
        # Load metadata from embeddings file
        self.metadata = []
        with open('recipes_for_embeddings.jsonl', 'r') as f:
            for line in f:
                self.metadata.append(json.loads(line))
        
        # Load full recipe details (with ingredients and directions)
        with open('full_format_recipes.json', 'r') as f:
            full_recipes = json.load(f)
        
        # HashTable for optimized lookup
        self.recipe_lookup = {r.get('title', '').strip(): r for r in full_recipes if r.get('title')}
        
        # Load embedding model
        print("Loading embedding model BAAI/bge-m3...")
        self.embed_model = SentenceTransformer('BAAI/bge-m3')
        
        # Load FAISS index
        print("Loading FAISS index...")
        self.index = faiss.read_index('recipe_index.faiss')
        
        print(f"✓ Recipe retrieval system loaded: {len(self.metadata)} recipes indexed")
    
    def forward(self, query: str) -> str:
        """Search for recipes matching the query.
        
        Args:
            query: Natural language search query
            
        Returns:
            Formatted string with recipe titles, ingredients, and directions
        """
        assert isinstance(query, str), "Query must be a string."
        
        # 1. Embed the Query and Ensure Float 32 (Required by FAISS)
        query_vec = self.embed_model.encode([query], convert_to_tensor=False)
        query_vec = np.array(query_vec).astype('float32')
        
        # 2. Normalize euclidean distance
        faiss.normalize_L2(query_vec)
        
        # 3. Search
        _, indices = self.index.search(query_vec, self.k)
        
        # 4. Retrieve
        retrieved_docs = [self.metadata[idx] for idx in indices[0] if idx != -1]
        
        # 5. Format Output
        output = f"Found {len(retrieved_docs)} recipes matching '{query}':\n\n"
        
        for i, doc in enumerate(retrieved_docs, 1):
            title = doc['title'].strip()
            full_recipe = self.recipe_lookup.get(title)
            
            output += f"{'='*40} Recipe {i} {'='*40}\n"
            output += f"TITLE: {title}\n"
            
            if full_recipe:
                output += "INGREDIENTS:\n" + "\n".join([f" - {ing}" for ing in full_recipe.get('ingredients', [])])
                output += "\n\nDIRECTIONS:\n" + "\n".join([f" {j}. {step}" for j, step in enumerate(full_recipe.get('directions', []), 1)])
            else:
                # Fallback
                output += f"SUMMARY: {doc.get('text_for_embedding', 'No details available')}"
            
            output += "\n\n"
            
        return output

In [3]:
# Initialize model and tool
model = InferenceClientModel("Qwen/Qwen2.5-14B-Instruct")
retrieval_tool = RecipeInfoRetrieverTool()

Loading recipe retrieval system...


'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 1569c44e-05f3-4382-bb48-6b11794a25ec)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-m3/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


Loading embedding model BAAI/bge-m3...
Loading FAISS index...
✓ Recipe retrieval system loaded: 18222 recipes indexed


In [4]:
result = retrieval_tool.forward("eggplant and garlic")
print(result)

Found 3 recipes matching 'eggplant and garlic':

======================================== Recipe 1 ========================================
TITLE: Roasted Eggplant and Garlic Dip
INGREDIENTS:
 - 2 small heads garlic
 - 1 eggplant (1 pound)
 - 1/4 cup extra-virgin olive oil
 - 1 teaspoon red-wine vinegar, or to taste

DIRECTIONS:
 1. Preheat oven to 425°F.
 2. Separate garlic cloves without peeling and tightly wrap together in foil. Prick eggplant with a fork. In a shallow baking pan roast garlic and eggplant in middle of oven until very tender, about 30 minutes for garlic and about 45 minutes for eggplant.
 3. Unwrap garlic and peel, transferring garlic to a food processor. Scrape flesh from eggplant into food processor, discarding skin. Purée mixture until smooth and, with motor running, add oil and vinegar until combined. Season dip with salt and pepper and serve with pita toasts.

======================================== Recipe 2 ========================================
TITLE: Herbe